<a href="https://colab.research.google.com/github/calisley/dpi-681/blob/main/section_4/Section_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# @title
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError
from getpass import getpass

try:
    key = userdata.get('OPENAI_API_KEY')
    print("OPENAI_API_KEY is set in Colab Secrets.")
except SecretNotFoundError:
    print("No OPENAI_API_KEY found in Colab Secrets.")
    print("Open the 'Secrets' tab on the left (the key icon')")
    print("Click 'Add new secret'")
    print("Set the name to OPENAI_API_KEY")
    print("Set the value to your API key")
    print("Click on the 'notebook access' toggle to turn it on")

OPENAI_API_KEY is set in Colab Secrets.


In [7]:
# @title
import os, json, requests
from typing import List, Dict, Any
import numpy as np
from tqdm import tqdm
from openai import OpenAI

from google.colab import userdata
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY missing from Colab Secrets. Rerun the secrets cell.")
client = OpenAI(api_key=OPENAI_API_KEY)

# Part 0: Square Function and Tool Use

In [44]:
def square(number):
    return number ** 2

In [34]:
tools = [{
    "type": "function",
    "name": "square",
    "parameters": {
        "type": "object",
        "properties": {
            "number": { "type": "number" }
        },
        "required": ["number"]
    }
}]

In [40]:
resp = client.responses.create(
        model="gpt-5-mini",
        input="What is 17^2? use any tools you have available",
        tools=tools
  )

In [50]:
for item in resp.output:
    if item.type == "function_call" and item.name == "square":
        args = json.loads(item.arguments)
        result = square(args['number'])  # equivalent to square(number=args["number"])

print(result)

289


# Part 1: Weather Agent

## First, we define the real "get_weather" function we will use

This uses the Open-meteo API, a free API that allows us to get weather for any latitute and longtitute.

In [60]:
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

## Then, we write the *schema* for the function.

It is okay if you do not understand this -- OpenAI has a tool to help us do this we will cover next time.

In [52]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "string",
                "description": "Latitude of desired location"
            },
            "longitude": {
                "type": "string",
                "description": "Longitude of desired location"
            }
        },
        "required": ["latitude","longitude"],
        "additionalProperties": False
    }
}]



## Then, we set up the chat loop:

In [55]:

# Step 2: Get user input
user_query = input("Ask the agent anything (weather or unrelated): ")

response = client.responses.create(
    model="gpt-5-mini",
    input=user_query,
    tools=tools
)

output = response.output[1]

if output.type == "function_call":
    print("Tool call detected!")
    tool_call = output
    args = json.loads(tool_call.arguments)
    result = get_weather(**args)
    print(f"Temperature: {result}°C")
else:
    print(response.output_text)

Ask the agent anything (weather or unrelated): what is the temperature in boston?



In [61]:
output = response.output[1]

if output.type == "function_call":
    print("Tool call detected!")
    tool_call = output
    args = json.loads(tool_call.arguments)
    result = get_weather(**args)
    print(f"Temperature: {result}°C")
else:
    print(response.output_text)

Tool call detected!
Temperature: 5.2°C


# Part 2: Agent with loops

In [19]:
import json
import requests
import pandas as pd

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

def square(number):
  return number**2


###### Model's version of the functions ######

tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "Latitude of desired location"
                },
                "longitude": {
                    "type": "string",
                    "description": "Longitude of desired location"
                }
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        }
    },
    {
    "type": "function",
    "name": "square",
    "parameters": {
        "type": "object",
        "properties": {
            "number": { "type": "number" }
        },
        "required": ["number"]
    }
    }
]


# General function for calling our functions
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)
    elif name == "square":
        return square(**args)
    else:
        return f"No such function: {name}"


###### UI ######

# Step 2: Get user input
user_query = input("Ask the agent something: ")
messages = [{"role": "user", "content": user_query}]

while True:
    response = client.responses.create(
        model="gpt-4.1",
        input=messages,
        tools=tools
    )
    output = response.output

    if output and output[0].type == "function_call":
        tool_call = output[0]
        print(f"\nTool call detected: {tool_call.name}")
        args = json.loads(tool_call.arguments)

        result = call_function(tool_call.name, args)

        if isinstance(result, dict):
            result = json.dumps(result)

        # Append both the call and the result
        messages.append(tool_call.model_dump())

        messages.append({
            "type": "function_call_output",
            "call_id": tool_call.call_id,
            "output": str(result)
        })

    else:
        print("\nAgent response:\n")
        print(response.output_text)
        break

Ask the agent something: What is the temperature in boston squared?

Tool call detected: get_weather

Tool call detected: square

Agent response:

The current temperature in Boston is 2.1°C. When squared, this gives you 4.41.


# Part 3: News sentiment plotter (if time)


Note this will not work if you do not have a Guardian API key. For more information on how to get a Guardian API key, click the link here: https://open-platform.theguardian.com/

In [17]:
from openai import OpenAI
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd

GUARDIAN_API_KEY = ""  # Replace with your Guardian API key

###### Defining real functions for US to exectute ######
def get_news_articles(query: str, order_by: str, page_size: int, output_path: str) -> str:
    url = "https://content.guardianapis.com/search"
    params = {
        "q": query,
        "order-by": order_by,
        "page-size": page_size,
        "show-fields": "bodyText",
        "api-key": GUARDIAN_API_KEY
    }

    response = requests.get(url, params=params)
    response.raise_for_status()

    data = response.json()
    results = data.get("response", {}).get("results", [])

    articles = []
    for item in results:
        fields = item.get("fields", {})
        body = fields.get("bodyText", "")
        date = item.get("webPublicationDate", "")
        articles.append({"date": date, "article_text": body})

    df = pd.DataFrame(articles)
    df.to_csv(output_path, index=False, encoding="utf-8")
    print(f"Saved {len(df)} articles to {output_path}")
    return df


def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

def analyze_sentiment(sentiments, output_path):
    try:
        df = pd.DataFrame(sentiments)

        if not all(1 <= s <= 10 for s in df["sentiment"]):
            return "Error: Sentiment scores must be between 1 and 10."

        df.to_csv(output_path, index=False, encoding="utf-8")

        print(f"Saved sentiment analysis to {output_path}")
        return f"Saved sentiment analysis to {output_path}"
    except Exception as e:
        return f"Error saving sentiment output: {str(e)}"

def graph_data(csv_path: str, x_column: str, y_column: str, title: str, x_label: str, y_label: str) -> str:
    try:
        df = pd.read_csv(csv_path)

        if x_column not in df.columns or y_column not in df.columns:
            return f"Error: Columns '{x_column}' and/or '{y_column}' not found in {csv_path}."

        # Convert date to full datetime
        df[x_column] = pd.to_datetime(df[x_column], errors='coerce')
        df = df.dropna(subset=[x_column, y_column])
        df = df.sort_values(by=x_column)

        # Plot each entry (no aggregation)
        plt.figure(figsize=(12, 6))
        plt.plot(df[x_column], df[y_column], marker='o', linestyle='-', alpha=0.7)

        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()

        output_image_path = csv_path.replace(".csv", "_graph.png")
        plt.savefig(output_image_path)
        plt.close()

        return f"Graph saved to {output_image_path}"

    except Exception as e:
        return f"Graphing error: {str(e)}"



###### Model's version of the functions ######

tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "Latitude of desired location"
                },
                "longitude": {
                    "type": "string",
                    "description": "Longitude of desired location"
                }
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        }
    },
    {
        "type": "function",
        "name": "get_news_articles",
        "description": "Fetch news articles from The Guardian API and save them to a CSV.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query, e.g., 'climate change'"
                },
                "order_by": {
                    "type": "string",
                    "enum": ["newest", "oldest", "relevance"],
                    "description": "How to sort the articles"
                },
                "page_size": {
                    "type": "integer",
                    "description": "How many articles to return (max 200)"
                },
                "output_path": {
                    "type": "string",
                    "description": "File path where the articles will be saved as CSV"
                }
            },
            "required": ["query", "order_by", "page_size", "output_path"],
            "additionalProperties": False
        }
    },
    {
        "type": "function",
        "name": "analyze_sentiment",
        "description": "Analyze sentiment of article texts on a scale from 1 (negative) to 10 (positive), and save results as a CSV.",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiments": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "datetime": {"type": "string"},
                            "sentiment": {
                                "type": "integer",
                                "enum": [1,2,3,4,5,6,7,8,9,10],
                                "description": "Sentiment score: 1 (very negative) to 10 (very positive)"
                            }
                        },
                        "required": ["datetime",  "sentiment"],
                        "additionalProperties": False
                    },
                    "description": "Each entry contains the article snippet, datetime, and its sentiment score. Do NOT change the datetimes."
                },
                "output_path": {
                    "type": "string",
                    "description": "CSV file path to save sentiment results"
                }
            },
            "required": ["sentiments", "output_path"],
            "additionalProperties": False
        },
        "strict": True
    },
    {
        "type": "function",
        "name": "graph_data",
        "description": "Generate a graph from a CSV file.",
        "parameters": {
            "type": "object",
            "properties": {
                "csv_path": {
                    "type": "string",
                    "description": "Path to the CSV file containing the data"
                },
                "x_column": {
                    "type": "string",
                    "description": "Column name to use as x-axis"
                },
                "y_column": {
                    "type": "string",
                    "description": "Column name to use as y-axis"
                },
                "title": {
                    "type": "string",
                    "description": "Title of the graph"
                },
                "x_label": {
                    "type": "string",
                    "description": "Label for the x-axis"
                },
                "y_label": {
                    "type": "string",
                    "description": "Label for the y-axis"
                }
            },
            "required": ["csv_path", "x_column", "y_column", "title", "x_label", "y_label"],
            "additionalProperties": False
        }
    }
]


# General function for calling our functions
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)
    elif name == "get_news_articles":
        return get_news_articles(**args)
    elif name == "graph_data":
        return graph_data(**args)
    elif name == "analyze_sentiment":
        return analyze_sentiment(**args)
    else:
        return f"No such function: {name}"


###### UI ######

# Step 2: Get user input
user_query = input("Ask the agent something: ")
messages = [{"role": "user", "content": user_query}]

while True:
    response = client.responses.create(
        model="gpt-4.1",
        input=messages,
        tools=tools
    )
    output = response.output

    if output and output[0].type == "function_call":
        tool_call = output[0]
        print(f"\nTool call detected: {tool_call.name}")
        args = json.loads(tool_call.arguments)

        result = call_function(tool_call.name, args)

        if isinstance(result, dict):
            result = json.dumps(result)

        # Append both the call and the result
        messages.append(tool_call.model_dump())

        messages.append({
            "type": "function_call_output",
            "call_id": tool_call.call_id,
            "output": str(result)
        })

    else:
        print("\nAgent response:\n")
        print(response.output_text)
        break

KeyboardInterrupt: Interrupted by user

In [10]:
response

Response(id='resp_092fb5c0dbab0c84006916a758f3e88190870910aeafb9945b', created_at=1763092313.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_092fb5c0dbab0c84006916a759e7e881908dab873369074c54', content=[ResponseOutputText(annotations=[], text='Here is a plot of the sentiment scores of the 10 most recent articles about Harvard over time:\n\n- The x-axis shows the dates of the articles.\n- The y-axis shows the sentiment scores (1 = very negative, 10 = very positive).\n\nWould you like to see or download the graph image?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'latitude': {'type': 'string', 'description': 'Latitude of desired location'}, 'longitude': {'type': 'string', 'description': 

In [16]:
messages

[{'role': 'user',
  'content': 'Generate me a plot of the sentiment scores of the 10 most recent articles about harvard over tine'},
 {'arguments': '{"query":"harvard","order_by":"newest","page_size":10,"output_path":"harvard_articles.csv"}',
  'call_id': 'call_tHqk68kUshoorFTl3xkndQLS',
  'name': 'get_news_articles',
  'type': 'function_call',
  'id': 'fc_092fb5c0dbab0c84006916a74f07848190b711f6162b963141',
  'status': 'completed'},
 {'type': 'function_call_output',
  'call_id': 'call_tHqk68kUshoorFTl3xkndQLS',
  'output': '                   date                                       article_text\n0  2025-11-12T17:29:42Z  In these turbulent times, we take small joys w...\n1  2025-11-12T17:20:47Z  A court in Beijing is expected to rule on Thur...\n2  2025-11-10T12:00:48Z  In the afterglow of electoral triumph, hope sp...\n3  2025-11-09T10:00:29Z  Across the country, Donald Trump’s crackdown o...\n4  2025-11-07T15:03:52Z  When Zohran Mamdani was elected as New York Ci...\n5  2025-11-07